# Aircraft Engine Predictive Maintenance Project

## Overview

In this project, we aim to develop a predictive maintenance model for aircraft engines. The primary objective is to predict the Remaining Useful Life (RUL) of these engines based on sensor readings and operational parameters. Predictive maintenance is crucial for optimizing maintenance schedules, reducing downtime, and ensuring the efficient operation of aircraft.

## Dataset

The dataset used in this project consists of sensor data and engine settings collected over time. It includes features such as setting parameters, sensor measurements.

## Approach

The project follows these main steps:

1. **Data Preparation:** Merging training and test datasets, calculating RUL, and defining feature columns.
2. **Model Selection:** Utilizing Linear Regression as the chosen model for predicting RUL.
3. **Training and Evaluation:** Training the model on the training set and evaluating its performance using metrics such as Mean Squared Error (MSE).

## Goals

- Develop a regression model to accurately predict the Remaining Useful Life of aircraft engines.
- Understand the significance of various features in determining engine health.
- Lay the groundwork for future explorations, potentially incorporating classification tasks for more nuanced predictions.

Let's delve into the details of each step and explore the outcomes of our predictive maintenance model.


## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report

## Loading Datasets
- **df_train, df_test, df_truth:** Reading datasets from Excel files that contain information about aircraft engine performance.


In [3]:
df_train = pd.read_excel(r'D:\\Projects\\BDA\\aircraft_reliability\\data\\PM_train.xlsx')
df_test = pd.read_excel(r'D:\\Projects\\BDA\\aircraft_reliability\\data\\PM_test.xlsx')
df_truth = pd.read_excel(r'D:\\Projects\\BDA\\aircraft_reliability\\data\\PM_truth.xlsx')

## Data Concatenation
- **Concatenation:** Combining `df_train` and `df_test` into a single dataframe (`df`) to work with the entire dataset.

In [4]:
df = pd.concat([df_train, df_test], ignore_index=True)
df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392.0,2388,100,39.06,23.4190
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392.0,2388,100,39.00,23.4236
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390.0,2388,100,38.95,23.3442
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392.0,2388,100,38.88,23.3739
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393.0,2388,100,38.90,23.4044


In [5]:
df.columns

Index(['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
       's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
       's15', 's16', 's17', 's18', 's19', 's20', 's21'],
      dtype='object')

# Feature Engineering


###    RUL

Remaining Useful Life (RUL) is a crucial concept in predictive maintenance, representing the estimated operational lifespan or cycles remaining for an asset before it is expected to fail. In the context of aircraft engines, RUL helps anticipate the point at which an engine may no longer meet performance requirements. By calculating RUL, organizations can proactively plan maintenance activities, optimize resource allocation, and minimize downtime. This predictive approach enables timely interventions, reducing the risk of unexpected failures and enhancing the overall efficiency and reliability of the asset. RUL serves as a key metric for making informed decisions in maintenance and operational strategies.
- **Remaining Useful Life (RUL):** Calculated by subtracting the current cycle from the maximum cycle for each engine.
- **Feature Columns:** Defined a list of feature columns for model training.

In [6]:
df['RUL'] = df.groupby('id')['cycle'].transform(max) - df['cycle']
feature_columns = ['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

C:\Users\anush\AppData\Local\Temp\ipykernel_3400\3507201732.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df['RUL'] = df.groupby('id')['cycle'].transform(max) - df['cycle']


## Merging Truth Data
- **Merge Operation:** Combining `df` with truth data (`df_truth`) based on the engine ID, providing additional information for analysis.

In [7]:
df = pd.merge(df, df_truth, on='id', how='left')

## Labeling
- **Threshold Labeling:** Creating a binary label ('label') based on a threshold of 30 for RUL, which helps in creating a classification problem.

In [8]:
threshold = 30
df['label'] = (df['RUL'] <= threshold).astype(int)

## Train-Test Split
- **Splitting Data:** Using the train-test split function to divide the data into training and testing sets. The model will be trained on the training set and evaluated on the testing set.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df[feature_columns], df['RUL'], test_size=0.2, random_state=42)

# Model Training
- **Linear Regression Model:** Creating and training a linear regression model using the feature columns defined earlier.

In [10]:
# Creating a linear regression model
model = LinearRegression()
# Train the model
model.fit(X_train, y_train)

LinearRegression()

## Prediction and Evaluation
- **Making Predictions:** Applying the trained model to predict RUL on the test set.
- **Binary Conversion:** Converting RUL values to binary labels (0 or 1) for evaluation as a classification problem.
- **Evaluation Metrics:** Calculating accuracy and classification report metrics for assessing the model's performance.

In [11]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# y_test contains the true labels and y_pred contains the predicted labels
y_test_binary = (y_test <= threshold).astype(int)
y_pred_binary = (y_pred <= threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test_binary, y_pred_binary)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test_binary, y_pred_binary))

Accuracy: 0.9455019957015659
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5883
           1       0.91      0.48      0.63       631

    accuracy                           0.95      6514
   macro avg       0.93      0.74      0.80      6514
weighted avg       0.94      0.95      0.94      6514



In [13]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

print(f"MAE: {mean_absolute_error(y_test,y_pred)}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test,y_pred))}")

MAE: 38.82815817967203
RMSE: 50.616039991969686


# Conclusion

In this project, we aimed to predict the Remaining Useful Life (RUL) of aircraft engines using a regression approach. The dataset, consisting of sensor readings and engine parameters, was utilized to train a Linear Regression model. The key steps in the project include:

1. **Data Preprocessing:** Merging training and test datasets, calculating RUL, and defining feature columns.
2. **Model Training:** A Linear Regression model was employed to predict the RUL based on the selected features.
3. **Evaluation:** The model was evaluated using metrics like Mean Squared Error (MSE) and accuracy for a specific threshold.

The Linear Regression model demonstrated promising results in predicting RUL. The accuracy and classification report indicate its effectiveness in estimating the remaining operational life of aircraft engines. However, it's important to note that no classification task was undertaken in this specific implementation.

Future work may involve exploring classification models to predict binary outcomes related to engine health, such as whether an engine will fail within a certain time frame. This would provide a more nuanced understanding of maintenance needs and further enhance the practical applications of predictive maintenance strategies.
